<a href="https://colab.research.google.com/github/thehydrowave/Adversarial-diffusion-distillation-pokemon/blob/main/Addgood1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
!pip install numpy==1.26.4 diffusers==0.25.0 transformers==4.37.2 accelerate==0.27.2 datasets==2.17.0 huggingface_hub===0.25.0 peft==0.7.1 --no-cache-dir --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 111.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 221.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 145.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 220.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 205.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 269.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 214.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 325.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 141.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 315.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import numpy as np
import torch
from datasets import load_dataset
from diffusers import StableDiffusionPipeline
from diffusers.models.attention_processor import LoRAAttnProcessor
from transformers import CLIPTokenizer
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm
from torchvision.io import read_image
from datasets import Dataset
import os
import torchvision.transforms as transforms
from diffusers import StableDiffusionPipeline


print("✅ NumPy:", np.__version__)
print("diffusers:", diffusers.__version__)
print("transformers:", transformers.__version__)
print("accelerate:", accelerate.__version__)
print("datasets:", datasets.__version__)
print("huggingface_hub:", huggingface_hub.__version__)

/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


✅ NumPy: 1.26.4


/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


NameError: name 'diffusers' is not defined

In [2]:
# 📥 Charger le dataset Naruto
dataset = load_dataset("lambdalabs/naruto-blip-captions", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [15]:
# Créer un dossier temporaire
os.makedirs("naruto_images", exist_ok=True)

processed = []

for i, example in enumerate(tqdm(dataset)):
    # Resize et sauvegarde de l'image
    image = example["image"].convert("RGB").resize((128, 128))  # Resize to 256x256
    image_path = f"naruto_images/img_{i}.png"
    image.save(image_path)

    processed.append({
        "image_path": image_path,
        "text": example["text"]
    })

# Créer un dataset Hugging Face basé sur les chemins
dataset_small = Dataset.from_list(processed)

from torchvision.io import read_image

def transform_from_path(example):
    image_tensor = read_image(example["image_path"]).float() / 255.0
    example["pixel_values"] = image_tensor
    return example

dataset_small = dataset_small.map(transform_from_path)
dataset_small = dataset_small.select(range(500))

100%|██████████| 1221/1221 [00:52<00:00, 23.47it/s]


Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [4]:
# 🔧 Charger le modèle SD + tokenizer
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.to("cuda")

tokenizer = pipe.tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
# 📦 Créer un DataLoader
def collate_fn(batch):
    # Ensure pixel_values are tensors before stacking
    images = torch.stack([torch.tensor(example["pixel_values"]) if isinstance(example["pixel_values"], list) else example["pixel_values"] for example in batch])
    prompts = [example["text"] for example in batch]
    inputs = tokenizer(prompts, padding="max_length", truncation=True, return_tensors="pt")
    return {
        "pixel_values": images,
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
    }

# Use dataset_small instead of dataset
train_dataloader = DataLoader(dataset_small, batch_size=1, shuffle=True, collate_fn=collate_fn)

In [6]:
from diffusers.models.attention_processor import LoRAAttnProcessor

# 1. Activer LoRA dans le pipeline
pipe.enable_lora()

# 2. Appliquer LoRA aux couches d’attention du UNet
attn_processors = {}
for name, processor in pipe.unet.attn_processors.items():
    try:
        lora_attn_proc = LoRAAttnProcessor(
            hidden_size=pipe.unet.config.block_out_channels[-1],
            cross_attention_dim=pipe.unet.config.cross_attention_dim
        )
        attn_processors[name] = lora_attn_proc
    except Exception as e:
        print(f"⛔ Skipping {name}: {e}")

pipe.unet.set_attn_processor(attn_processors)
print(f"✅ LoRA appliqué à {len(attn_processors)} couches.")

# 3. Vérification des poids LoRA
lora_params = [p for n, p in pipe.unet.named_parameters() if "lora" in n and p.requires_grad]

if not lora_params:
    raise ValueError("❌ Aucun poids LoRA détecté.")
else:
    print(f"🧠 Nombre de paramètres LoRA entraînables : {sum(p.numel() for p in lora_params)}")
    for name, param in pipe.unet.named_parameters():
        if "lora" in name:
            print(f"   🔹 {name} | requires_grad = {param.requires_grad}")

# 4. Préparer l'optimiseur
optimizer = torch.optim.Adam(lora_params, lr=1e-4)



✅ LoRA appliqué à 0 couches.
🧠 Nombre de paramètres LoRA entraînables : 1179648
   🔹 down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor.to_q_lora.down.weight | requires_grad = True
   🔹 down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor.to_q_lora.up.weight | requires_grad = True
   🔹 down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor.to_k_lora.down.weight | requires_grad = True
   🔹 down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor.to_k_lora.up.weight | requires_grad = True
   🔹 down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor.to_v_lora.down.weight | requires_grad = True
   🔹 down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor.to_v_lora.up.weight | requires_grad = True
   🔹 down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor.to_out_lora.down.weight | requires_grad = True
   🔹 down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor.to_out_lora.up.weight | requires_grad = True
   🔹 down_bl

In [14]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [16]:
# 🔁 Boucle d'entraînement
for epoch in range(1):
    loop = tqdm(train_dataloader, desc=f"Epoch {epoch}")
    for step, batch in enumerate(loop):
        pixel_values = batch["pixel_values"].type(torch.float16).to("cuda")
        input_ids = batch["input_ids"].to("cuda")
        attention_mask = batch["attention_mask"].to("cuda")

        noise = torch.randn_like(pixel_values)
        noised_images = pixel_values + noise

        timesteps = torch.randint(0, pipe.scheduler.config.num_train_timesteps, (pixel_values.shape[0],)).long().to("cuda").type(torch.float16)
        encoder_hidden_states = pipe.text_encoder(input_ids.to("cuda"))[0].type(torch.float16)

        noised_images = torch.cat([noised_images, torch.zeros_like(noised_images[:, :1, :, :])], dim=1)

        for param in pipe.unet.parameters():
            param.data = param.data.type(torch.float16)

        with torch.cuda.amp.autocast(dtype=torch.float16):
            model_output = pipe.unet(noised_images, timesteps, encoder_hidden_states).sample
            loss = model_output.mean()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loop.set_postfix(loss=loss.item())

Epoch 0:   0%|          | 0/1221 [00:00<?, ?it/s]<ipython-input-16-85bcbce3d90d>:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float16):
Epoch 0:   0%|          | 0/1221 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.00 GiB. GPU 0 has a total capacity of 39.56 GiB of which 3.03 GiB is free. Process 72164 has 36.52 GiB memory in use. Of the allocated memory 35.48 GiB is allocated by PyTorch, and 556.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# 💾 Sauvegarde des poids LoRA après entraînement
output_dir = "./naruto_lora_weights"
# Get LoRA layers from the pipeline
unet_lora_layers = pipe.unet.attn_processors

# Extract the LoRA weight tensors from the AttnProcessor2_0 objects
for key, attn_processor in unet_lora_layers.items():
    if isinstance(attn_processor, LoRAAttnProcessor):  # Only save LoRA weights
        # Save LoRA weights directly using the attn_processor's state_dict
        attn_processor.save_pretrained(os.path.join(output_dir, key))